<a href="https://colab.research.google.com/github/Cinnes8850/Energy_Infrastructure/blob/main/two_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install
!pip install -q pyomo
!pip install gurobipy

In [ ]:
# load libraries and solver
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition
import sys
sys.meta_path = [hook for hook in sys.meta_path if hasattr(hook, "find_spec")]
import gurobipy as gp  # import the installed package

In [ ]:
# create PYOMO optimization model
mdl = ConcreteModel()

In [ ]:
# define lines (i,j): (min_flow, max_flow)
line_data = {(1, 2): [-10, 10]}

In [ ]:
# Sets
mdl.Consumers = Set(initialize=["k1", "k2"])
mdl.Generators = Set(initialize=["g1", "g2"])

In [ ]:
# additional sets
mdl.Nodes = Set(initialize=[1, 2])
mdl.Lines = Set(initialize=line_data.keys(), dimen=2)

In [ ]:
# We can check the whole model with:
mdl.pprint()

4 Set Declarations
    Consumers : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'k1', 'k2'}
    Generators : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'g1', 'g2'}
    Lines : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :    1 : {(1, 2),}
    Nodes : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

4 Declarations: Consumers Generators Nodes Lines


In [ ]:
# Bid prices for demand and supply as nested Python dictionaries
# (we can use also Param() to create a parameter with Pyomo)
price_dem = {1: {"k1": 30, "k2":  5},
             2: {"k1": 50, "k2":  0}}
price_gen = {1: {"g1": 20, "g2": 40},
             2: {"g1":  5, "g2": 10}}

In [ ]:
# nested dictionary must be accessed using both keys, as follows:
print(price_dem[1]["k1"])

30


In [ ]:
# Max quantity for demand and supply as nested Python dictionaries
max_dem = {1: {"k1": 100, "k2": 100},
           2: {"k1":  80, "k2":  10}}
max_gen = {1: {"g1":  30, "g2": 100},
           2: {"g1":  20, "g2": 150}}

In [ ]:
# Variables for demand and generation
mdl.dem = Var(mdl.Nodes, mdl.Consumers, domain=NonNegativeReals)
mdl.gen = Var(mdl.Nodes, mdl.Generators, domain=NonNegativeReals)

In [ ]:
# we can check individual components
mdl.dem.pprint()
mdl.gen.pprint()

dem : Size=4, Index=Nodes*Consumers
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 'k1') :     0 :  None :  None : False :  True : NonNegativeReals
    (1, 'k2') :     0 :  None :  None : False :  True : NonNegativeReals
    (2, 'k1') :     0 :  None :  None : False :  True : NonNegativeReals
    (2, 'k2') :     0 :  None :  None : False :  True : NonNegativeReals
gen : Size=4, Index=Nodes*Generators
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 'g1') :     0 :  None :  None : False :  True : NonNegativeReals
    (1, 'g2') :     0 :  None :  None : False :  True : NonNegativeReals
    (2, 'g1') :     0 :  None :  None : False :  True : NonNegativeReals
    (2, 'g2') :     0 :  None :  None : False :  True : NonNegativeReals


In [ ]:
# power flow variable
mdl.f = Var(mdl.Lines, domain=Reals)

# check
mdl.f.pprint()

f : Size=1, Index=Lines
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 2) :  None :  None :  None : False :  True :  Reals


In [ ]:
# Constraints
def dem_max_rule(self, n, k):
    return mdl.dem[n,k] <= max_dem[n][k]
mdl.dem_max_constrait = Constraint(mdl.Nodes, mdl.Consumers, rule=dem_max_rule)

# check
mdl.dem_max_constrait.pprint()

dem_max_constrait : Size=4, Index=Nodes*Consumers, Active=True
    Key       : Lower : Body      : Upper : Active
    (1, 'k1') :  -Inf : dem[1,k1] : 100.0 :   True
    (1, 'k2') :  -Inf : dem[1,k2] : 100.0 :   True
    (2, 'k1') :  -Inf : dem[2,k1] :  80.0 :   True
    (2, 'k2') :  -Inf : dem[2,k2] :  10.0 :   True


In [ ]:
def gen_max_rule(self, n, g):
    return mdl.gen[n,g] <= max_gen[n][g]
mdl.gen_max_constrait = Constraint(mdl.Nodes, mdl.Generators, rule=gen_max_rule)

# check
mdl.gen_max_constrait.pprint()

gen_max_constrait : Size=4, Index=Nodes*Generators, Active=True
    Key       : Lower : Body      : Upper : Active
    (1, 'g1') :  -Inf : gen[1,g1] :  30.0 :   True
    (1, 'g2') :  -Inf : gen[1,g2] : 100.0 :   True
    (2, 'g1') :  -Inf : gen[2,g1] :  20.0 :   True
    (2, 'g2') :  -Inf : gen[2,g2] : 150.0 :   True


In [ ]:
tot_dem = sum(mdl.dem[1, k] for k in mdl.Consumers)
tot_gen = sum(mdl.gen[1, g] for g in mdl.Generators)
mdl.power_balance_1 = Constraint(expr=tot_dem - tot_gen + mdl.f[(1,2)] == 0)

In [ ]:
tot_dem = sum(mdl.dem[2, k] for k in mdl.Consumers)
tot_gen = sum(mdl.gen[2, g] for g in mdl.Generators)
mdl.power_balance_2 = Constraint(expr=tot_dem - tot_gen - mdl.f[(1,2)] == 0)

In [ ]:
# check
mdl.power_balance_1.pprint()
mdl.power_balance_2.pprint()

power_balance_1 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                     : Upper : Active
    None :   0.0 : dem[1,k1] + dem[1,k2] - (gen[1,g1] + gen[1,g2]) + f[1,2] :   0.0 :   True
power_balance_2 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                     : Upper : Active
    None :   0.0 : dem[2,k1] + dem[2,k2] - (gen[2,g1] + gen[2,g2]) - f[1,2] :   0.0 :   True


In [ ]:
# min flow
def min_flow_rule(self, i,j):
    return line_data[(i,j)][0] <= mdl.f[i,j]
mdl.min_flow = Constraint(mdl.Lines, rule=min_flow_rule)

# check
mdl.min_flow.pprint()

min_flow : Size=1, Index=Lines, Active=True
    Key    : Lower : Body   : Upper : Active
    (1, 2) : -10.0 : f[1,2] :  +Inf :   True


In [ ]:
# max flow
def max_flow_rule(self, i,j):
    return mdl.f[(i,j)] <= line_data[(i,j)][1]
mdl.max_flow = Constraint(mdl.Lines, rule=max_flow_rule)

# check
mdl.max_flow.pprint()

max_flow : Size=1, Index=Lines, Active=True
    Key    : Lower : Body   : Upper : Active
    (1, 2) :  -Inf : f[1,2] :  10.0 :   True


In [ ]:
# Objective function
total_benefit_consumers = sum(price_dem[n][k] * mdl.dem[n,k] for k in mdl.Consumers for n in mdl.Nodes)
total_cost_generators = sum(price_gen[n][k] * mdl.gen[n,k] for k in mdl.Generators for n in mdl.Nodes)

mdl.obj = Objective(expr=total_benefit_consumers - total_cost_generators, sense=maximize)

# check
mdl.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 30*dem[1,k1] + 50*dem[2,k1] + 5*dem[1,k2] + 0*dem[2,k2] - (20*gen[1,g1] + 5*gen[2,g1] + 40*gen[1,g2] + 10*gen[2,g2])


In [ ]:
# check the whole model
mdl.pprint()

4 Set Declarations
    Consumers : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'k1', 'k2'}
    Generators : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'g1', 'g2'}
    Lines : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :    1 : {(1, 2),}
    Nodes : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

3 Var Declarations
    dem : Size=4, Index=Nodes*Consumers
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 'k1') :     0 :  None :  None : False :  True : NonNegativeReals
        (1, 'k2') :     0 :  None :  None : False :  True : NonNegativeReals
        (2, 'k1') :     0 :  None :  None : False :  True : NonNegativeReals
        (2, 'k2') :     0 :

In [ ]:
# We have to tell Pyomo that we want dual variables
# In Pyomo, the existence of an active Suffix with the name "dual"
# that has an import style suffix direction will cause constraint
# dual information to be collected into the solver results
# (assuming the solver supplies dual information).
mdl.dual = Suffix(direction=Suffix.IMPORT)

In [ ]:
# Create an object representing the solver, in this case gurobi
solver = SolverFactory("gurobi")

In [ ]:
# Cplex options in interactive format
# solver.options["timelimit"] = 1800    # time limit in seconds

# solver.options["simplex tolerances optimality"] = 1e-9  # optimality tolerance
# solver.options["simplex tolerances feasibility"] = 1e-9 # feasibility tolerance

# useful for MILP problems, i.e. involving binary variables
# solver.options["mip tolerances integrality"] = 0        # integrality tolerance
# solver.options["mip tolerances mipgap"] = 0    # mixed integer optimality gap tolerance
# solver.options["mip tolerances absmipgap"] = 0 # absolute mixed integer optimality gap tolerance

# solve the optimization problem
results = solver.solve(mdl, tee=True)

Read LP format model from file /tmp/tmpiv3i3ok9.pyomo.lp
Reading time = 0.00 seconds
x1: 12 rows, 9 columns, 20 nonzeros
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
QCPDual  1

Optimize a model with 12 rows, 9 columns and 20 nonzeros
Model fingerprint: 0x761b018c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 11 rows and 4 columns
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9000000e+03   2.625000e+01   0.000000e+00      0s
       1    3.8000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work

In [ ]:
# other useful arguments
# solve(mdl, tee=True, keepfiles=True, symbolic_solver_labels=True)
# solve(mdl, tee=True, keepfiles=False, warmstart=True)
# solve(mdl, tee=True, keepfiles=True, solnfile=file.sol)
# solve(mdl, tee=True, keepfiles=False, warmstart=True, warmstart_file=file.sol)

# ALWAYS check solver's termination condition
if results.solver.termination_condition != TerminationCondition.optimal:
    raise Exception
else:
    print(results.solver.status)
    print(results.solver.termination_condition)
    print(results.solver.termination_message)
    # print(results.solver.time)


ok
optimal
Model was solved to optimality (subject to tolerances), and an optimal solution is available.


In [ ]:
# print solved variables
mdl.dem.pprint()
mdl.gen.pprint()

# access single values
print("demand k1 at node 1=", mdl.dem[1, "k1"].value)

# print objective function value
print("welfare=",value(mdl.obj))

# print the market prices,
# i.e. the dual variable of
# the power balance constraints
print("price at node 1=%.2f" % mdl.dual[mdl.power_balance_1])
print("price at node 2=%.2f" % mdl.dual[mdl.power_balance_2])

dem : Size=4, Index=Nodes*Consumers
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 'k1') :     0 :  40.0 :  None : False : False : NonNegativeReals
    (1, 'k2') :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 'k1') :     0 :  80.0 :  None : False : False : NonNegativeReals
    (2, 'k2') :     0 :   0.0 :  None : False : False : NonNegativeReals
gen : Size=4, Index=Nodes*Generators
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 'g1') :     0 :  30.0 :  None : False : False : NonNegativeReals
    (1, 'g2') :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 'g1') :     0 :  20.0 :  None : False : False : NonNegativeReals
    (2, 'g2') :     0 :  70.0 :  None : False : False : NonNegativeReals
demand k1 at node 1= 40.0
welfare= 3800.0
price at node 1=30.00
price at node 2=10.00
